In [1]:
from query.models import FaceCharacterActor, Shot
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.parsers import in_array, bbox_payload_parser, merge_dict_parsers, dict_payload_parser
from rekall.merge_ops import payload_plus
from rekall.payload_predicates import payload_satisfies
from rekall.spatial_predicates import scene_graph
from esper.rekall import intrvllists_to_result_bbox
from query.models import Face
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.parsers import in_array, bbox_payload_parser
from rekall.merge_ops import payload_plus, merge_named_payload, payload_second
from esper.rekall import intrvllists_to_result_bbox
from rekall.payload_predicates import payload_satisfies
from rekall.list_predicates import length_at_most
from rekall.logical_predicates import and_pred, or_pred, true_pred
from rekall.spatial_predicates import scene_graph, make_region
from rekall.temporal_predicates import before, after, overlaps
from rekall.bbox_predicates import height_at_least
from esper.rekall import intrvllists_to_result, intrvllists_to_result_with_objects, add_intrvllists_to_result
from esper.prelude import esper_widget
from rekall.interval_list import Interval, IntervalList

In [2]:
RIGHT_HALF_MIN_X = 0.45
LEFT_HALF_MAX_X = 0.55
MIN_FACE_HEIGHT = 0.4
MAX_FACES_ON_SCREEN = 2
# faces are sampled every 12 frames
SAMPLING_RATE = 12
ONE_SECOND = 1
FOUR_SECONDS = 96
TEN_SECONDS = 240

In [3]:
# Annotate face rows with start and end frames and the video ID
faces_with_character_actor_qs = FaceCharacterActor.objects.annotate(
    min_frame=F('face__frame__number'),
    max_frame=F('face__frame__number'),
    video_id=F('face__frame__video_id'),
    bbox_x1=F('face__bbox_x1'),
    bbox_y1=F('face__bbox_y1'),
    bbox_x2=F('face__bbox_x2'),
    bbox_y2=F('face__bbox_y2'),
    character_name=F('characteractor__character__name')
)

In [4]:
faces_with_identity = VideoIntervalCollection.from_django_qs(
    faces_with_character_actor_qs,
    with_payload=in_array(merge_dict_parsers([
        bbox_payload_parser(VideoIntervalCollection.django_accessor),
        dict_payload_parser(VideoIntervalCollection.django_accessor, { 'character': 'character_name' }),
    ]))
).coalesce(payload_merge_op=payload_plus)

In [5]:
shots_qs = Shot.objects.filter(
    labeler=Labeler.objects.get(name='shot-hsvhist-face'))
shots = VideoIntervalCollection.from_django_qs(shots_qs)

In [6]:
def payload_unique_characters(payload1, payload2):
    if 'characters' not in payload1[0]:
        unique_characters = set([p['character'] for p in payload1])
        for p in payload2:
            unique_characters.add(p['character'])
        payload1[0]['characters'] = list(unique_characters)
    else:
        unique_characters = set([p['character'] for p in payload2])
        unique_characters.update(payload1[0]['characters'])
        payload1[0]['characters'] = list(unique_characters)
    return payload1
    
    

In [7]:
shots_with_faces = shots.merge(faces_with_identity, 
                               predicate=overlaps(), 
                               payload_merge_op=payload_second)

In [8]:
shots_with_faces = shots_with_faces.coalesce(payload_merge_op=payload_unique_characters)


In [9]:
def cross_product_faces(intrvl1, intrvl2):
    payload1 = intrvl1.get_payload()
    payload2 = intrvl2.get_payload()
    chrtrs1 = payload1[0]['characters'] if 'characters' in payload1[0] else list(set([p['character'] for p in payload1]))
    chrtrs2 = payload2[0]['characters'] if 'characters' in payload2[0] else list(set([p['character'] for p in payload2]))
    new_intervals = []
    for i in payload1:
        for j in chrtrs2:
            if i!=j:
                new_payload = {'A': i, 'B': j}
#                 new_payload.update()
                start = min(intrvl1.start, intrvl2.start)
                end = max(intrvl1.end, intrvl2.end)
#                 print(intrvl1.keys())
#                 print(intrvl1.video_id == intrvl2.video_id )
                new_intervals.append(Interval(start, end, {'A': i, 'B': j}))

    return new_intervals

In [10]:
def faces_equal(payload1, payload2):
    return (payload1['A'] == payload2['A'] and payload1['B'] == payload2['B']) or (payload1['A'] == payload2['B'] and payload1['B'] == payload2['A'])

In [ ]:
def faces_equal(payload1, payload2):
    if type(payload1) is not list and type(payload1) is not list:
        return (payload1['A'] == payload2['A'] and payload1['B'] == payload2['B']) or (payload1['A'] == payload2['B'] and payload1['B'] == payload2['A'])
    elif type(payload1) is list and type(payload1) is list:
        for i in payload1:
            for j in payload2:
                if i['A'] == j['A'] and i['B'] == j['B']:
                    return True
    elif type(payload1) is list:
        for i in payload1:
            if i['A'] == payload2['A'] and i['B'] == payload2['B']:
                return True
    else:
        for i in payload2:
            if i['A'] == payload1['A'] and i['B'] == payload1['B']:
                return True
    return False

def times_equal(intrvl1, intrvl2):
    return intrvl.start == intervl2.start and intrvl.end == intervl2.end

def merge_to_list(payload1, payload2):
    p1 = payload1 if type(payload1) is list else [payload1]
    p2 = payload2 if type(payload2) is list else [payload2]
    return p1+p2

In [11]:
two_shots = shots_with_faces.join(shots_with_faces, predicate=after(max_dist=ONE_SECOND, min_dist=ONE_SECOND), 
                                merge_op=cross_product_faces)


In [12]:
num_intervals = 0
for video_id in two_shots.intervals.keys():
    intvllist = two_shots.get_intervallist(video_id)
    s = intvllist.size()
    print(s)
    num_intervals += s
print(num_intervals)

401
48
2150
931
763
134
424
356
1191
706
965
627
743
874
537
552
483
1212
262
385
507
1035
679
260
903
1412
672
1091
739
464
1208
659
794
922
785
366
802
624
570
608
938
1911
1061
1493
937
185
1093
70
651
684
1388
950
1225
841
1012
439
506
506
1704
1168
972
1547
874
602
1336
1632
528
602
1564
1185
707
1049
1527
485
612
62828


In [13]:
conversations = two_shots.coalesce(predicate=payload_satisfies(faces_equal, arity=2))

In [14]:
num_intervals = 0
for video_id in conversations.intervals.keys():
    intvllist = conversations.get_intervallist(video_id)
    s = intvllist.size()
    print(s)
    num_intervals += s
print(num_intervals)

401
48
2150
931
763
134
424
1191
706
965
627
743
874
537
552
483
385
262
1212
507
1035
679
260
903
1336
1412
672
1091
739
464
1208
659
794
922
1547
802
841
624
570
608
890
1911
1061
1493
937
185
1093
70
684
1388
950
1225
785
1012
972
506
528
707
1632
1168
506
366
874
602
439
356
651
602
1564
1185
1704
1049
1527
485
612
62780


In [75]:
scene = three_shot.merge(three_shot, predicate=and_pred(after(max_dist=ONE_SECOND, min_dist=ONE_SECOND), 
                                              payload_satisfies(check_B_intersects, arity=2), arity=2)).coalesce()#, payload_merge_op=updateA))

In [15]:
esper_widget(intrvllists_to_result_with_objects(
            conversations.get_allintervals(), lambda payload, video: []),
            crop_bboxes=False,
            disable_playback=False,
            jupyter_keybindings=False)

VGridWidget(jsglobals={'queries': [['All faces', 'def all_faces():\n    from query.models import Face\n    fro…

In [ ]:
conversations.get_allintervals().key()

In [ ]:
conversations.intrvls.keys()

In [ ]:
conversations

In [13]:
def conversationsq():
    from query.models import FaceCharacterActor, Shot
    from rekall.video_interval_collection import VideoIntervalCollection
    from rekall.parsers import in_array, bbox_payload_parser, merge_dict_parsers, dict_payload_parser
    from rekall.merge_ops import payload_plus
    from rekall.payload_predicates import payload_satisfies
    from rekall.spatial_predicates import scene_graph
    from esper.rekall import intrvllists_to_result_bbox
    from query.models import Face
    from rekall.video_interval_collection import VideoIntervalCollection
    from rekall.parsers import in_array, bbox_payload_parser
    from rekall.merge_ops import payload_plus, merge_named_payload, payload_second
    from esper.rekall import intrvllists_to_result_bbox
    from rekall.payload_predicates import payload_satisfies
    from rekall.list_predicates import length_at_most
    from rekall.logical_predicates import and_pred, or_pred, true_pred
    from rekall.spatial_predicates import scene_graph, make_region
    from rekall.temporal_predicates import before, after, overlaps
    from rekall.bbox_predicates import height_at_least
    from esper.rekall import intrvllists_to_result, intrvllists_to_result_with_objects, add_intrvllists_to_result
    from esper.prelude import esper_widget
    from rekall.interval_list import Interval, IntervalList
    
    RIGHT_HALF_MIN_X = 0.45
    LEFT_HALF_MAX_X = 0.55
    MIN_FACE_HEIGHT = 0.4
    MAX_FACES_ON_SCREEN = 2
    # faces are sampled every 12 frames
    SAMPLING_RATE = 12
    ONE_SECOND = 1
    FOUR_SECONDS = 96
    TEN_SECONDS = 240


    faces_with_character_actor_qs = FaceCharacterActor.objects.annotate(
        min_frame=F('face__frame__number'),
        max_frame=F('face__frame__number'),
        video_id=F('face__frame__video_id'),
        bbox_x1=F('face__bbox_x1'),
        bbox_y1=F('face__bbox_y1'),
        bbox_x2=F('face__bbox_x2'),
        bbox_y2=F('face__bbox_y2'),
        character_name=F('characteractor__character__name')
    ).filter(face__frame__video__name__contains="godfather part iii")
    
    faces_with_identity = VideoIntervalCollection.from_django_qs(
        faces_with_character_actor_qs,
        with_payload=in_array(merge_dict_parsers([
            bbox_payload_parser(VideoIntervalCollection.django_accessor),
            dict_payload_parser(VideoIntervalCollection.django_accessor, { 'character': 'character_name' }),
        ]))
    ).coalesce(payload_merge_op=payload_plus)

    shots_qs = Shot.objects.filter(
        labeler=Labeler.objects.get(name='shot-hsvhist-face'))
    shots = VideoIntervalCollection.from_django_qs(shots_qs)
    
    def payload_unique_characters(payload1, payload2):
        if 'characters' not in payload1[0]:
            unique_characters = set([p['character'] for p in payload1])
            for p in payload2:
                unique_characters.add(p['character'])
            payload1[0]['characters'] = list(unique_characters)
        else:
            unique_characters = set([p['character'] for p in payload2])
            unique_characters.update(payload1[0]['characters'])
            payload1[0]['characters'] = list(unique_characters)
        return payload1
        
    shots_with_faces = shots.merge(faces_with_identity, 
                                  predicate=overlaps(), 
                                  payload_merge_op=payload_second)
                                   
    shots_with_faces = shots_with_faces.coalesce(payload_merge_op=payload_unique_characters)

    def cross_product_faces(intrvl1, intrvl2):
        payload1 = intrvl1.get_payload()
        payload2 = intrvl2.get_payload()
        chrtrs1 = payload1[0]['characters'] if 'characters' in payload1[0] else list(set([p['character'] for p in payload1]))
        chrtrs2 = payload2[0]['characters'] if 'characters' in payload2[0] else list(set([p['character'] for p in payload2]))
        new_intervals = []
        for i in chrtrs1:
            for j in chrtrs2:
                if i!=j:
                    new_payload = {'A': i, 'B': j}
    #                 new_payload.update()
                    start = min(intrvl1.start, intrvl2.start)
                    end = max(intrvl1.end, intrvl2.end)
    #                 print(intrvl1.keys())
    #                 print(intrvl1.video_id == intrvl2.video_id )
                    new_intervals.append(Interval(start, end, {'A': i, 'B': j}))

        return new_intervals
        
    def faces_equal(payload1, payload2):
        p1 = [payload1]
        if type(payload1) is dict and 'chrs' in payload1:
            p1 = payload1['chrs']
        elif type(payload1) is list:
            p1 = payload1
        
        p2 = [payload2]
        if type(payload2) is dict and 'chrs' in payload2:
            p2 = payload2['chrs']
        elif type(payload2) is list:
            p2 = payload2
            
        payload1 = p1
        payload2 = p2
        
        if type(payload1) is not list and type(payload1) is not list:
            return (payload1['A'] == payload2['A'] and payload1['B'] == payload2['B']) or (payload1['A'] == payload2['B'] and payload1['B'] == payload2['A'])
        elif type(payload1) is list and type(payload2) is list:
            for i in payload1:
                for j in payload2:
                    if i['A'] == j['A'] and i['B'] == j['B']:
                        return True
                    if i['A'] == j['B'] and i['B'] == j['A']:
                        return True
        elif type(payload1) is list:
            for i in payload1:
                if i['A'] == payload2['A'] and i['B'] == payload2['B']:
                    return True
                if i['A'] == payload2['B'] and i['B'] == payload2['A']:
                    return True
        else:
            for i in payload2:
                if i['A'] == payload1['A'] and i['B'] == payload1['B']:
                    return True
                if i['A'] == payload1['B'] and i['B'] == payload1['A']:
                    return True
        return False

    def times_equal(intrvl1, intrvl2):
        return (intrvl1.start >= intrvl2.start and intrvl1.end <= intrvl2.end) or (intrvl2.start >= intrvl1.start and intrvl2.end <= intrvl1.end)
        
    def times_overlap(intrvl1, intrvl2):
        return intrvl1.start <= intrvl2.end and intrvl2.start <= intrvl1.end
    
    def merge_to_list(payload1, payload2):
        p1 = payload1 if type(payload1) is list else [payload1]
        p2 = payload2 if type(payload2) is list else [payload2]
        return p1+p2
    
    def count_shots(payload1, payload2):
        p1 = [payload1]
        if type(payload1) is dict and 'chrs' in payload1:
            p1 = payload1['chrs']
        elif type(payload1) is list:
            p1 = payload1
        
        p2 = [payload2]
        if type(payload2) is dict and 'chrs' in payload2:
            p2 = payload2['chrs']
        elif type(payload2) is list:
            p2 = payload2
        
        p1_shots = payload1['shots'] if type(payload1) is dict and 'shots' in payload1 else 1
        p2_shots = payload2['shots'] if type(payload2) is dict and 'shots' in payload2 else 1
#         p1 = payload1['chrs'] if type(payload1) is dict else payload1
#         p2 = payload2['chrs'] if type(payload2) is dict else payload2
        return {'shots': p1_shots + p2_shots, 'chrs': p1 + p2}

    two_shots = shots_with_faces.join(shots_with_faces, predicate=after(max_dist=ONE_SECOND, min_dist=ONE_SECOND), 
                                merge_op=cross_product_faces)

    # convs = two_shots.coalesce(predicate=or_pred(times_equal,payload_satisfies(faces_equal, arity=2), arity=2), payload_merge_op=merge_to_list)
    convs = two_shots.coalesce(predicate=times_equal, payload_merge_op=merge_to_list)
    convs = convs.coalesce(predicate=payload_satisfies(faces_equal, arity=2), payload_merge_op=count_shots)
    
    def filter_fn(intvl):
        payload = intvl.get_payload()
        if type(payload) is dict and 'shots' in payload:
            return payload['shots'] >= 2
        return False 
    
    convs = convs.filter(filter_fn)
    convs = convs.coalesce(predicate=times_overlap)
    # convs = two_shots.coalesce(predicate=payload_satisfies(faces_equal, arity=2))
    # We use pose_to_dict to draw PoseWrapper objects.
    for video_id in convs.intervals.keys():
        print(video_id)
        intvllist = convs.get_intervallist(video_id)
        for intvl in intvllist.get_intervals():
            print(intvl.payload)
            print(str(intvl.start) + ':' + str(intvl.end))
    
    return intrvllists_to_result_with_objects(
            convs.get_allintervals(), lambda payload, video: [])

FN=conversationsq
FN()

216
{'chrs': [{'B': 'mary corleone', 'A': 'don michael corleone'}, {'B': 'kay adams michelson', 'A': 'don michael corleone'}, {'B': 'anthony vito corleone', 'A': 'don michael corleone'}, {'B': 'don michael corleone', 'A': 'mary corleone'}, {'B': 'don michael corleone', 'A': 'anthony vito corleone'}, {'B': 'mary corleone', 'A': 'don michael corleone'}, {'B': 'anthony vito corleone', 'A': 'don michael corleone'}], 'shots': 3}
3241:4282
{'chrs': [{'B': 'don michael corleone', 'A': 'kay adams michelson'}, {'B': 'b.j. harrison', 'A': 'kay adams michelson'}, {'B': 'mary corleone', 'A': 'kay adams michelson'}, {'B': 'anthony vito corleone', 'A': 'kay adams michelson'}, {'B': 'kay adams michelson', 'A': 'don michael corleone'}, {'B': 'kay adams michelson', 'A': 'b.j. harrison'}, {'B': 'kay adams michelson', 'A': 'archbishop gilday'}, {'B': 'kay adams michelson', 'A': 'connie corleone rizzi'}], 'shots': 2}
7877:8347
{'chrs': [{'B': 'vincent mancini', 'A': 'mary corleone'}, {'B': 'mary corleone'

{'count': 1,
 'result': [{'elements': [{'max_frame': 4282,
     'min_frame': 3241,
     'objects': [],
     'video': 216}],
   'label': '',
   'type': 'flat'},
  {'elements': [{'max_frame': 8347,
     'min_frame': 7877,
     'objects': [],
     'video': 216}],
   'label': '',
   'type': 'flat'},
  {'elements': [{'max_frame': 13454,
     'min_frame': 12481,
     'objects': [],
     'video': 216}],
   'label': '',
   'type': 'flat'},
  {'elements': [{'max_frame': 14225,
     'min_frame': 13673,
     'objects': [],
     'video': 216}],
   'label': '',
   'type': 'flat'},
  {'elements': [{'max_frame': 14729,
     'min_frame': 14226,
     'objects': [],
     'video': 216}],
   'label': '',
   'type': 'flat'},
  {'elements': [{'max_frame': 15595,
     'min_frame': 15095,
     'objects': [],
     'video': 216}],
   'label': '',
   'type': 'flat'},
  {'elements': [{'max_frame': 16189,
     'min_frame': 15596,
     'objects': [],
     'video': 216}],
   'label': '',
   'type': 'flat'},
  {'elem

In [2]:
data = [
    (8757,9049),
    (12750,13463),
    (13683,14227),
    (21357,22236),
    (22294,22758),
    (23147,25854),
    (26007,26942),
    (27620,28172),
    (28382,28623),
    (28785,29036),
    (29904,31014),
    (33936,35339),
    (35421,36248),
    (39388,40062),
    (41675,42689),
    (51246,52118),
    (53117,54776),
    (54895,55762),
    (56819,59963),
    (60253,61875),
    (66533,67846),
    (68729,69040),
    (69421,70153),
    (70285,71102)]
intrvllist = IntervalList([Interval(start, end, payload=None) for (start,end) in data])
shot_reverse_shot_labelled = {216: intrvllist}
esper_widget(intrvllists_to_result_with_objects(shot_reverse_shot_labelled, lambda payload, video: []), disable_captions=True)

VGridWidget(jsglobals={'queries': [['All faces', 'def all_faces():\n    from query.models import Face\n    fro…